# VoxelOps Validation Framework - Introduction

This notebook introduces the VoxelOps validation framework, which provides:
- **Pre-validation**: Check inputs before running procedures
- **Post-validation**: Verify outputs after procedures complete
- **Audit logging**: Complete trail of validation and execution events
- **Smart failure handling**: Early exit with detailed error messages

## Table of Contents
1. [Basic Usage](#basic-usage)
2. [Understanding Results](#understanding-results)
3. [Validation Reports](#validation-reports)
4. [Audit Logs](#audit-logs)
5. [Error Handling](#error-handling)

## Setup

In [2]:
from pathlib import Path

from voxelops import (
    QSIPrepInputs,
    run_procedure,
)

# Example paths (adjust for your data)
bids_dir = Path("/media/storage/yalab-dev/qsiprep_test/BIDS")
output_dir = Path("/media/storage/yalab-dev/qsiprep_test/derivatives")
log_dir = Path("/media/storage/yalab-dev/qsiprep_test/logs")

## 1. Basic Usage <a id="basic-usage"></a>

The simplest way to use validation is with `run_procedure()`:

In [3]:
# Define inputs as usual
inputs = QSIPrepInputs(
    bids_dir=bids_dir,
    participant="CLMC10",
    # session="01",  # Optional
)

# Run with validation (replaces run_qsiprep)
result = run_procedure(
    "qsiprep",
    inputs,
    log_dir=log_dir,  # Where to save audit logs
)

# Check success
if result.success:
    print(f"✓ Success! Completed in {result.duration_seconds:.1f}s")
else:
    print(f"✗ Failed: {result.get_failure_reason()}")

✗ Failed: Pre-validation failed: Found 0 DWI files, required 1


### Old vs New API

```python
# OLD: Direct runner (no validation)
from voxelops import run_qsiprep
result = run_qsiprep(inputs)  # Returns dict

# NEW: With validation framework
from voxelops import run_procedure
result = run_procedure("qsiprep", inputs)  # Returns ProcedureResult
```

**Both APIs work!** The old API remains for backward compatibility.

## 2. Understanding Results <a id="understanding-results"></a>

`run_procedure()` returns a `ProcedureResult` object with:

In [4]:
# Status information
print(f"Status: {result.status}")  # success, pre_validation_failed, etc.
print(f"Success: {result.success}")  # True/False
print(f"Duration: {result.duration_seconds}s")

# Procedure info
print(f"\nProcedure: {result.procedure}")
print(f"Participant: {result.participant}")
print(f"Session: {result.session}")
print(f"Run ID: {result.run_id}")

# Validation reports (if run)
if result.pre_validation:
    print(f"\nPre-validation: {result.pre_validation.summary()}")
if result.post_validation:
    print(f"Post-validation: {result.post_validation.summary()}")

# Audit log location
print(f"\nAudit log: {result.audit_log_file}")

Status: pre_validation_failed
Success: False
Duration: 0.00128s

Procedure: qsiprep
Participant: CLMC10
Session: None
Run ID: ed0f1221-35ac-4d85-a211-937e6414c8f8

Pre-validation: PRE validation FAILED: 2 passed, 4 errors, 0 warnings

Audit log: /media/storage/yalab-dev/qsiprep_test/logs/sub-CLMC10_qsiprep_324abf8d-02a3-466b-8bf6-df3d668216d6.jsonl


### Possible Status Values

| Status | Meaning |
|--------|--------|
| `success` | Everything passed |
| `pre_validation_failed` | Inputs invalid, didn't run procedure |
| `execution_failed` | Procedure crashed/failed |
| `post_validation_failed` | Outputs missing/invalid |

## 3. Validation Reports <a id="validation-reports"></a>

Each validation phase produces a detailed report:

In [5]:
# Get pre-validation report
pre_report = result.pre_validation

if pre_report:
    print(f"Phase: {pre_report.phase}")  # 'pre' or 'post'
    print(f"Passed: {pre_report.passed}")
    print(f"Total checks: {len(pre_report.results)}")

    # Errors (severity='error' and failed)
    print(f"\nErrors: {len(pre_report.errors)}")
    for error in pre_report.errors:
        print(f"  - {error.rule_description}: {error.message}")

    # Warnings (severity='warning' and failed)
    print(f"\nWarnings: {len(pre_report.warnings)}")
    for warning in pre_report.warnings:
        print(f"  - {warning.rule_description}: {warning.message}")

    # Passed checks
    print(f"\nPassed: {len(pre_report.passed_checks)}")
    for check in pre_report.passed_checks:
        print(f"  ✓ {check.rule_description}")

Phase: pre
Passed: False
Total checks: 6

Errors: 4
  - Verify DWI files exist (pattern: dwi/*_dwi.nii.gz): Found 0 DWI files, required 1
  - Verify b-value files exist (pattern: dwi/*_dwi.bval): Found 0 b-value files, required 1
  - Verify b-vector files exist (pattern: dwi/*_dwi.bvec): Found 0 b-vector files, required 1
  - Verify T1w anatomical exist (pattern: anat/*_T1w.nii.gz): Found 0 T1w anatomical, required 1

Warnings: 0

Passed: 2
  ✓ Verify BIDS directory directory exists
  ✓ Verify participant exists in input directory


### Inspecting Individual Results

Each result contains detailed information:

In [ ]:
# Look at first error in detail
if pre_report and pre_report.errors:
    error = pre_report.errors[0]

    print(f"Rule: {error.rule_name}")
    print(f"Description: {error.rule_description}")
    print(f"Severity: {error.severity}")
    print(f"Message: {error.message}")
    print("\nDetails:")
    for key, value in error.details.items():
        print(f"  {key}: {value}")
    print(f"\nTimestamp: {error.timestamp}")

## 4. Audit Logs <a id="audit-logs"></a>

Every procedure run creates a JSONL audit log with all events:

In [ ]:
import json

# Read the audit log
audit_file = Path(result.audit_log_file)
if audit_file.exists():
    with open(audit_file) as f:
        events = [json.loads(line) for line in f]

    print(f"Total events: {len(events)}\n")

    # Show each event
    for event in events:
        print(f"[{event['timestamp']}] {event['event_type']}")
        if event.get('data'):
            print(f"  Data: {list(event['data'].keys())}")

### Event Types

| Event Type | When It Occurs |
|------------|----------------|
| `procedure_start` | Beginning of run |
| `pre_validation` | Pre-validation passed |
| `pre_validation_failed` | Pre-validation failed |
| `execution_start` | Starting procedure |
| `execution_success` | Procedure completed |
| `execution_failed` | Procedure crashed |
| `post_validation` | Post-validation passed |
| `post_validation_failed` | Post-validation failed |
| `procedure_complete` | Everything succeeded |
| `procedure_failed` | Something failed |

## 5. Error Handling <a id="error-handling"></a>

The framework provides helpful error messages:

In [ ]:
def run_and_handle_errors(procedure, inputs):
    """Example of comprehensive error handling."""
    result = run_procedure(procedure, inputs)

    if result.success:
        print(f"✓ Success! Duration: {result.duration_seconds:.1f}s")
        return result

    # Handle different failure modes
    if result.status == "pre_validation_failed":
        print("✗ Pre-validation failed - inputs are invalid")
        print(f"\nReason: {result.get_failure_reason()}")

        # Show all errors
        for error in result.pre_validation.errors:
            print(f"  • {error.message}")
            # Show details for debugging
            if error.details:
                print(f"    Details: {error.details}")

    elif result.status == "execution_failed":
        print("✗ Procedure execution failed")
        print(f"\nError: {result.execution.get('error')}")

    elif result.status == "post_validation_failed":
        print("✗ Post-validation failed - outputs are invalid")
        print(f"\nReason: {result.get_failure_reason()}")

        for error in result.post_validation.errors:
            print(f"  • {error.message}")

    return result

# Example usage
# result = run_and_handle_errors("qsiprep", inputs)

## Saving to Database

The result is designed for easy database storage:

In [ ]:
# Convert to dictionary for database
result_dict = result.to_dict()

# Show structure
print("Keys:", list(result_dict.keys()))

# Example fields
print(f"\nProcedure: {result_dict['procedure']}")
print(f"Status: {result_dict['status']}")
print(f"Success: {result_dict['success']}")
print(f"Duration: {result_dict['duration_seconds']}")
print(f"\nPre-validation passed: {result_dict['pre_validation']['passed'] if result_dict['pre_validation'] else 'N/A'}")
print(f"Post-validation passed: {result_dict['post_validation']['passed'] if result_dict['post_validation'] else 'N/A'}")

# Save to database (example)
# db.procedures.insert_one(result_dict)

## Skipping Validation

You can skip validation phases if needed:

In [ ]:
# Skip pre-validation (NOT RECOMMENDED)
result = run_procedure(
    "qsiprep",
    inputs,
    skip_pre_validation=True,  # Skips input checks
)

# Skip post-validation
result = run_procedure(
    "qsiprep",
    inputs,
    skip_post_validation=True,  # Skips output checks
)

# Skip both (basically same as old run_qsiprep)
result = run_procedure(
    "qsiprep",
    inputs,
    skip_pre_validation=True,
    skip_post_validation=True,
)

## Next Steps

See procedure-specific notebooks:
- `02_heudiconv_validation.ipynb` - DICOM to BIDS conversion
- `03_qsiprep_validation.ipynb` - DWI preprocessing
- `04_qsirecon_validation.ipynb` - DWI reconstruction
- `05_qsiparc_validation.ipynb` - Parcellation
- `06_custom_validation_rules.ipynb` - Writing custom validators